In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN as GCN_DR
from deeprobust.graph.targeted_attack import FGA
from scipy.sparse import csr_matrix

In [2]:
from greedy_mcmc_attack import *
from collections import defaultdict

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
cora_dataset = Planetoid(root='', name='Pubmed')
data = cora_dataset[0].to(device)
print(data)
idx_train, idx_val, idx_test = data.train_mask.nonzero(as_tuple=True)[0].tolist(), data.val_mask.nonzero(as_tuple=True)[0].tolist(), data.test_mask.nonzero(as_tuple=True)[0].tolist()

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [4]:
data_dr = Dataset(root='/tmp/', name='pubmed', setting='nettack')
adj, features, labels = data_dr.adj, data_dr.features, data_dr.labels
# idx_train, idx_val, idx_test = data_dr.idx_train, data_dr.idx_val, data_dr.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading pubmed dataset...


In [5]:
degrees = np.array(adj.sum(axis=1)).flatten()
max_degree_vertex = degrees.argmax()

In [6]:
adj = csr_matrix(adj)

In [7]:
surrogate = GCN_DR(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [8]:
ptb_rate = 0.1

In [ ]:
model = FGA(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
model = model.to(device)
perturbations = int(ptb_rate * (adj.sum() // 2))
model.attack(features, adj, labels, idx_train, max_degree_vertex, n_perturbations=perturbations)
modified_adj = model.modified_adj

### arch tests

In [ ]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [ ]:
model_save_path = "../models/pubmed_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges.pth" # pubmed doesn't exist yet... have to satiate

In [ ]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [ ]:
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

In [ ]:
from scipy.sparse import csr_matrix

adj1 = data_dr.adj.tocsr()
adj2 = modified_adj.tocsr()

arr_diff = (adj1 - adj2).tocoo()

In [ ]:
arr_diff

In [ ]:
data_modded = copy.deepcopy(data)

In [ ]:
# thanks gpt
existing = data_modded.edge_index.t().cpu().numpy()

diff_coo = arr_diff.tocoo()
diff_edges = np.stack([diff_coo.row, diff_coo.col], axis=1)

existing_set = set(map(tuple, existing))
diff_set = set(map(tuple, diff_edges))
diff_set_full = diff_set | set((v, u) for u, v in diff_set)

cleaned_edges = existing_set - diff_set_full
print(len(cleaned_edges))

added_edges = diff_set_full - existing_set
print(len(added_edges))

final_edges = cleaned_edges | added_edges

final_edges = torch.tensor(list(final_edges), dtype=torch.long).t().to(device)
data_modded.edge_index = final_edges

In [ ]:
new_loss, new_accuracy = train.test(data_modded)
# print(f"New Accuracy: {new_accuracy}")
# print(f"New Loss: {new_loss}")
print(f"Evasion Change in Accuracy: {(new_accuracy - initial_accuracy):.4f}")
print(f"Evasion Change in Loss: {(new_loss - initial_loss):.4f}")

### poisoning
model_poisoning = GCN(data_modded.x.shape[1], cora_dataset.num_classes, [16]).to(device)
model.reset_parameters()
train = Trainable(model)
train.fit(data_modded, 1000)

poisoning_accuracy, poisoning_loss = train.test(data_modded)
print(f"Poisoning Change in Accuracy: {(poisoning_accuracy - initial_accuracy):.4f}")
print(f"Poisoning Change in Loss: {(poisoning_loss - initial_loss):.4f}")